In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\akk\Clubcorp\CommunityBenefit


In [2]:
import pandas as pd
import sqlite3

In [3]:
golf_benefit_df = pd.read_csv("output\intermediate\Community_Golf_Benefits.csv")  

In [4]:
#benefit_df.columns

<b>Split benefit column into two columns (a) benefit_1 (b) advance</b>

In [5]:
# Max split = 1
golf_benefit_df['benefit_1'], golf_benefit_df['advance'] = golf_benefit_df['benefit'].str.split(' # ', 1).str

In [6]:
golf_benefit_df.count()

community             142
benefit_package_id    142
benefit_id            142
benefit               142
benefit_1             142
advance               142
dtype: int64

In [7]:
golf_benefit_metadata_df = pd.read_csv("datasets\metadata\golf_benefit_meta_data.csv")
golf_advance_metadata_df = pd.read_csv("datasets\metadata\golf_advance_meta_data.csv")

In [8]:
golf_benefit_metadata_df.head()

,benefit,UsageType,IsQuantityUnlimited,Quantity,Period,Scope,MaxPerClub
0,CFO - unlimited,CFO,Yes,NaN,Per Month,Per Club,NaN
1,"Unlimited access, no fees",CFO,Yes,NaN,Per Month,Per Club,NaN
2,Unlimited CFO,CFO,Yes,NaN,Per Month,Per Club,NaN
3,Unlimited cfo golf,CFO,Yes,NaN,Per Month,Per Club,NaN
4,1 CFO per club,CFO,NaN,1.0,Per Month,Per Club,NaN


In [9]:
golf_advance_metadata_df.head(20)

,Advance,Prompt,PromptScope,DefaultValue,Prompt1,PromptScope1,DefaultValue1
0,7 day,Advance Tee Times,All Days,7 days,NaN,NaN,NaN
1,5 day,Advance Tee Times,All Days,5 days,NaN,NaN,NaN
2,30 day,Advance Tee Times,All Days,30 days,NaN,NaN,NaN
3,30 days,Advance Tee Times,All Days,30 days,NaN,NaN,NaN
4,72 hour,Advance Tee Times,All Days,72 hours,NaN,NaN,NaN
5,48 hour,Advance Tee Times,All Days,48 hours,NaN,NaN,NaN
6,72 hours,Advance Tee Times,All Days,72 hours,NaN,NaN,NaN
7,72 hour advance,Advance Tee Times,All Days,72 hours,NaN,NaN,NaN
8,5 day advance,Advance Tee Times,All Days,5 days,NaN,NaN,NaN
9,8 DAY,Advance Tee Times,All Days,8 days,NaN,NaN,NaN


In [10]:
%%time
conn = sqlite3.connect("golfBenefit.db")
cur = conn.cursor()
golf_benefit_df.to_sql("golfBenefit", conn, if_exists="replace")
golf_benefit_metadata_df.to_sql("golfBenefitMetadata", conn, if_exists="replace")
golf_advance_metadata_df.to_sql("advanceMetadata", conn, if_exists="replace")

Wall time: 188 ms


In [11]:
pd.read_sql_query("select count(distinct benefit_id) from golfBenefit", conn)

,count(distinct benefit_id)
0,142


In [12]:
query = "select golfBenefit.community, golfBenefit.benefit_package_id, \
         golfBenefit.benefit_id, \
         golfBenefit.benefit_1 as benefit, \
         golfBenefit.advance, \
         golfBenefitMetadata.UsageType,\
         golfBenefitMetadata.IsQuantityUnlimited,\
         golfBenefitMetadata.Quantity,\
         golfBenefitMetadata.Period,\
         golfBenefitMetadata.Scope, \
         golfBenefitMetadata.MaxPerClub, \
         advanceMetadata.Prompt, \
         advanceMetadata.PromptScope, \
         advanceMetadata.DefaultValue,\
         advanceMetadata.Prompt1, \
         advanceMetadata.PromptScope1, \
         advanceMetadata.DefaultValue1\
         from golfBenefit \
         left join golfBenefitMetadata on trim(upper(golfBenefitMetadata.benefit)) = trim(upper(golfBenefit.benefit_1)) \
         left join advanceMetadata on trim(upper(advanceMetadata.Advance)) = trim(upper(golfBenefit.advance))"
rs_benefit = pd.read_sql_query(query, conn)

In [13]:
print("Total number of golf benefits: " + str(len(rs_benefit)))

Total number of golf benefits: 142


<b>Save Golf Benefits With Additional Columns<b>

In [14]:
rs_benefit.to_csv("output/Community_Golf_Benefit_Extended.csv",index=False)

In [15]:
conn.execute("DROP TABLE golfBenefit")
conn.execute("DROP TABLE golfBenefitMetadata")
conn.execute("DROP TABLE advanceMetadata")
cur.close()
conn.close()